## Part I: Preprocessing
In this part, we will crop images, make masks and split them into 676 (26 x 26) boxes. Start by importing functions. 
Set info of the images for analysis. Sections to change are labelled in comments

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import f_preprocessing 
import f_modelDetection
import f_coordFinder
import f_intensityMeasurement
import f_distributions

wt = "WT"
mutant = "ND6" # change this to your mutant type 
conditions = [wt, mutant]

repeats = [1,2,3]

markers = ["DAPI", "SOX2", "BRA", "GATA3"] # change this to your markers 

channel_folders = { # change this to your markers and how theyre arranged on the CZI channels
        0: 'DAPI',
        1: 'SOX2',
        2: 'GATA3',
        3: 'BRA'
    }

directory = "CHIP_REPEATS_NEW" # directory as shown above
cropping_csv_path = "cropping.csv" # where all cropping info will be saved


Set where the czi files are saved in and other file directories. Arrange your CZI files in this format:

```
CHIP_REPEATS
├── 1 
│   ├── WT.czi
│   └── ND6.czi
├── 2
│   ├── WT.czi
│   └── ND6.czi
├── 3
│   ├── WT.czi
│   └── ND6.czi
```

Where the numbers represent your repeats and ND6 us what you set mutant as above 

### If Masks were made before cropping:
If you made the masks using the Raw image from the microscope then follow these steps.
1. Make sure they are named and inside the directory set above like this:
```
    CHIP_REPEATS
    ├── 1 
    │   ├── {condition}_{marker}_mask.tiff
    │   ├── WT_DAPI_mask.tiff
    │   ├── ND6_DAPI_mask.tiff
    │   ├── WT_SOX2_mask.tiff
    │   ├── ND6_SOX2_mask.tiff
    │   └── ...
    ├── 2
    │   ├── ...
    │   └── ...
    ├── 3
    │   ├──...
    │   └── ...
```
2. Convert CZI image into a tiff with the channels still intact using the block below

In [ ]:
f_preprocessing.convert_czi_to_tiff(directory)

After the step above step, you should use ImageJ/ Fiji to do these steps:
1. Manually crop and adjust angles 
2. Save coordinates for cropping in directory. (shown below)
2. Scale the cropped image to 7800 x 7800 pixels
3. Save the cropped scaled image in directory like this:

```
    CHIP_REPEATS
    ├── 1 
    │   ├── {condition}_scaled.tiff 
    │   ├── {condition}_coords.csv 
    │   ├── WT_scaled.tiff
    │   ├── ND6_scaled.tiff
    │   ├── WT_coords.csv
    │   └── ND6_coords.csv
    ├── 2
    │   ├── ...
    │   └── ...
    ├── 3
    │   ├──...
    │   └── ...
```

A tutorial of how to do the above steps right here: (link). 

Once youre done, Run the block below which:
1. Split the Tiffs into channels according to the channel folders set above 
2. Compile cropping coordinates into a cropping.csv file. 

After running the block below, if the raw images were rotated in ImageJ (before cropping), the angle for adjustment needs to be added to the csv file (default is 0).

In [ ]:
# Check the dimensions of your tiff files
# f_preprocessing.check_dimensions(directory) # it should show your scaled images have dimensions ({number of channels}, 7800, 7800)
f_preprocessing.check_dimensions("CHIP_REPEATS_NEW/2") # it should show your scaled images have dimensions ({number of channels}, 7800, 7800)

In [ ]:
# split Tiffs into channels 
f_preprocessing.split_tiff_into_channels(directory, repeats, conditions, channel_folders)

# Compile cropping coordinates (to crop masks according to how images were cropped)
f_preprocessing.compile_crop_coordinates(directory, repeats, conditions, cropping_csv_path)

Crop and resize masks. They will replace the original mask tiff. Then we check if all tiffs we have in our directory are the same size.

In [ ]:
f_preprocessing.crop_masks(directory, cropping_csv_path, conditions, repeats, markers)
print("finished cropping all masks!")
f_preprocessing.check_dimensions(directory)

Now all Images and Masks are aligned and the same size, we will split them into 676 boxes using a 26 x 26 grid. This process might take awhile. (7-10min)

In [ ]:
f_modelDetection.make_grid_and_split_all(directory, markers, conditions, repeats)

Manually select which boxes to include in the analysis. When finishing, all coordinates and diameters of models will be saved and you can move on to Radius adjustment.

In [ ]:
model_marker = ["DAPI"]
f_modelDetection.detect_blob_all(directory, model_marker, conditions, repeats)

## Part II: Radius Adjustment
In this part, we will use Repeat 1, DAPI to manually adjust and set the radius sizes for our Bins. Set values in the adjusting_values dictionary to save a folder of images in this format "{directory}/{repeat}/adjusting/{marker_adjusting}" of the marker along with a circle rperesenting the radius you set. DAPI is used to adjust for the Radius of the model. BRA is used to adjust for the middle radius, SOX2 is used to adjust for the smallest Radius. 

In [5]:
radius_values = {"DAPI": 110, # change these numbers (theyre just an example)
                     "BRA": 73.923,
                     "SOX2": 51.789 }


Run the block below to start adjustment. Making binary masks will take ~1min. Then it will make images saved in your directory under a folder called adjusting for you to see what the radius you set above looks like on the images (3-4min). Coordinates found will be saved in a folder called 'coordinates' in your directory. 

adjusting=False means no images will be saved (makes it run faster). adjusting = True makes it save all images to adjust for radius (takes a long time but useful for adjusting radius or trying to visualize how the radius looks like on the image / how well coordinate detection is going)

In [ ]:
f_coordFinder.run_R2(directory, repeats, conditions, radius_values, adjusting=True)


## Part III: Bin Intensity Measurement
In this part, using the radius we have adjusted and set above, we will load the coordinates saved and measure the RAW intensity for each marker (yes including DAPI). These raw coordinates will be saved in an npz in a folder called 'intensities'. Before we do that, lets make a mask for GATA3 to remove noise as the image is really noisy! from using pink nailpolish >:(

In [3]:
f_intensityMeasurement.make_GATA3_filter(directory, repeats, conditions)

------------Repeat: 1, Condition: WT, marker: GATA3
Saved GATA3 filter mask for Repeat: 1, Condition: WT
Bluriness for this set is 2566.054790282592
------------Repeat: 1, Condition: ND6, marker: GATA3
Saved GATA3 filter mask for Repeat: 1, Condition: ND6
Bluriness for this set is 5037.696607964865
------------Repeat: 2, Condition: WT, marker: GATA3
Saved GATA3 filter mask for Repeat: 2, Condition: WT
Bluriness for this set is 46710.808627789615
------------Repeat: 2, Condition: ND6, marker: GATA3
Saved GATA3 filter mask for Repeat: 2, Condition: ND6
Bluriness for this set is 50906.06651010813
------------Repeat: 3, Condition: WT, marker: GATA3
Saved GATA3 filter mask for Repeat: 3, Condition: WT
Bluriness for this set is 348463.63149315893
------------Repeat: 3, Condition: ND6, marker: GATA3
Saved GATA3 filter mask for Repeat: 3, Condition: ND6
Bluriness for this set is 272798.4865091042


In [6]:
f_intensityMeasurement.intensities_per_marker(directory, repeats, conditions, markers, radius_values)

------------Repeat: 1, Condition: WT, marker: DAPI
saved to: CHIP_REPEATS_NEW/1/intensities/inner/DAPI_WT CHIP_REPEATS_NEW/1/intensities/mid/DAPI_WT CHIP_REPEATS_NEW/1/intensities/outer/DAPI_WT
------------Repeat: 1, Condition: WT, marker: SOX2
saved to: CHIP_REPEATS_NEW/1/intensities/inner/SOX2_WT CHIP_REPEATS_NEW/1/intensities/mid/SOX2_WT CHIP_REPEATS_NEW/1/intensities/outer/SOX2_WT
------------Repeat: 1, Condition: WT, marker: BRA
saved to: CHIP_REPEATS_NEW/1/intensities/inner/BRA_WT CHIP_REPEATS_NEW/1/intensities/mid/BRA_WT CHIP_REPEATS_NEW/1/intensities/outer/BRA_WT
------------Repeat: 1, Condition: WT, marker: GATA3
saved to: CHIP_REPEATS_NEW/1/intensities/inner/GATA3_WT CHIP_REPEATS_NEW/1/intensities/mid/GATA3_WT CHIP_REPEATS_NEW/1/intensities/outer/GATA3_WT
------------Repeat: 1, Condition: ND6, marker: DAPI
saved to: CHIP_REPEATS_NEW/1/intensities/inner/DAPI_ND6 CHIP_REPEATS_NEW/1/intensities/mid/DAPI_ND6 CHIP_REPEATS_NEW/1/intensities/outer/DAPI_ND6
------------Repeat: 1, Con

Now we will normalize all the markers we have by DAPI.

In [7]:
f_intensityMeasurement.normalize_markers(directory, repeats, conditions, markers)

saving to csv for DAPI
------------------------Now processing:
repeat: 1
condition: 1
level: outer
marker: SOX2
loading intensities for SOX2 and DAPI
0-----------------------
marker 158.24321212702677
DAPI norm 3751.5076753333974
normalized 0.042181231073441336
1-----------------------
marker 96.77119009929486
DAPI norm 3345.2337506595672
normalized 0.02892808016187653
2-----------------------
marker 18.149034082738122
DAPI norm 2764.9214323378555
normalized 0.006564032478634433
3-----------------------
marker 0.0
DAPI norm 2493.4248237494603
normalized 0.0
4-----------------------
marker 41.64551737377616
DAPI norm 3189.293338452678
normalized 0.01305791376154222
5-----------------------
marker 219.48026662830287
DAPI norm 4146.310986428811
normalized 0.052933865150655214
6-----------------------
marker 13.925663037743993
DAPI norm 3169.6219845570954
normalized 0.004393477552084144
7-----------------------
marker 89.65857403320219
DAPI norm 3061.465358410901
normalized 0.0292861631724

You might want to take a quick look at meta_intensities.csv to see if your values kind of make sense. If you're satisfied, its time to visualize the data! (plot it!). You can find all plots in your directory, in a folder called 'plots'.

In [9]:
meta_path = f'{directory}/meta_intensities.csv'


for repeat in repeats:
    save_dir = f'{directory}/{repeat}/plots'
    f_intensityMeasurement.bar_plot(meta_path, repeat, plot_type='line', save_dir=save_dir)

for repeat in repeats:
    for marker in markers:
        if marker != "DAPI":
            save_dir = f'{directory}/{repeat}/plots'
            f_intensityMeasurement.plot_marker_condition_overlap(meta_path, repeat, marker, save_dir=save_dir)

Saved: CHIP_REPEATS_NEW/1/plots\intensities_plot_ND6.png
Saved: CHIP_REPEATS_NEW/1/plots\intensities_plot_WT.png
Saved: CHIP_REPEATS_NEW/2/plots\intensities_plot_ND6.png
Saved: CHIP_REPEATS_NEW/2/plots\intensities_plot_WT.png
Saved: CHIP_REPEATS_NEW/3/plots\intensities_plot_ND6.png
Saved: CHIP_REPEATS_NEW/3/plots\intensities_plot_WT.png
Saved plot: CHIP_REPEATS_NEW/1/plots\overlap_plot_SOX2.png
AUC WT: 0.335
AUC ND6: 0.098
Overlap AUC: 0.098
Overlap %: 45.31%
Saved plot: CHIP_REPEATS_NEW/1/plots\overlap_plot_BRA.png
AUC WT: 1.417
AUC ND6: 1.029
Overlap AUC: 0.864
Overlap %: 70.63%
Saved plot: CHIP_REPEATS_NEW/1/plots\overlap_plot_GATA3.png
AUC WT: 0.180
AUC ND6: 0.253
Overlap AUC: 0.173
Overlap %: 79.90%
Saved plot: CHIP_REPEATS_NEW/2/plots\overlap_plot_SOX2.png
AUC WT: 0.685
AUC ND6: 0.564
Overlap AUC: 0.548
Overlap %: 87.72%
Saved plot: CHIP_REPEATS_NEW/2/plots\overlap_plot_BRA.png
AUC WT: 0.893
AUC ND6: 1.672
Overlap AUC: 0.813
Overlap %: 63.40%
Saved plot: CHIP_REPEATS_NEW/2/plots\

## Part IV: Diameter and Intensity Distribution 
In this section, we will find the diameter (by DAPI) and Intensity (of each marker) of each gastruloid. Then we will plot this data on an overlap density plot (WT vs mutant). So the plots we will make are
1.  Gastruloid Diameter
2.  All markers Intensity 

We are trying to find a relationship between correct marker expression localisation so we will plot a scatter plot where the x-axis is the gastruloid diameter and the y-axis is the intensity of specific markers in specific regions. Here, we are using:
- SOX2: inner bin 
- BRA: middle bin
- GATA3: outer bin


In [10]:
f_distributions.get_distributions(directory, repeats, conditions, markers)

Starting Repeat: 1, condiiton: WT
loading coordinates...
Trying to load: CHIP_REPEATS_NEW/1/coordinates/WT.npz
loading regions...
Trying to load: CHIP_REPEATS_NEW/1/regions/WT.npz
loading binary masks...
Trying to load: CHIP_REPEATS_NEW/1/binarymasks/WT.npz
Geting Raw Distribution for Marker: DAPI
Geting Raw Distribution for Marker: SOX2
Geting Raw Distribution for Marker: BRA
Geting Raw Distribution for Marker: GATA3
Finished getting raw distributions for Repeat: 1, condiiton: WT
Normalizing data for Marker: SOX2
Fixed CSV saved to: CHIP_REPEATS_NEW/1/distribution/WT_SOX2.csv
Normalized data added and saved to: CHIP_REPEATS_NEW/1/distribution/WT_SOX2.csv
Normalizing data for Marker: BRA
Fixed CSV saved to: CHIP_REPEATS_NEW/1/distribution/WT_BRA.csv
Normalized data added and saved to: CHIP_REPEATS_NEW/1/distribution/WT_BRA.csv
Normalizing data for Marker: GATA3
Fixed CSV saved to: CHIP_REPEATS_NEW/1/distribution/WT_GATA3.csv
Normalized data added and saved to: CHIP_REPEATS_NEW/1/distri

In [17]:
marker_loc_dict = {'SOX2': 'inner', 'BRA': 'mid', 'GATA3': 'outer'}

for repeat in repeats:
    for marker, _ in marker_loc_dict.items():
            repeats_data = [
                (directory, repeat, 'WT', f'{directory}/{repeat}/intensities/meta_individual_WT.csv', f'{directory}/{repeat}/distribution/WT_DAPI.csv', marker),
                (directory, repeat, 'ND6', f'{directory}/{repeat}/intensities/meta_individual_ND6.csv', f'{directory}/{repeat}/distribution/ND6_DAPI.csv', marker),
            ]

            # f_distributions.combined_plot_intensity_vs_diameter(
            #     repeats_data,
            #     marker_loc_dict
            # )
            
            f_distributions.combined_plot_intensity_vs_DAPIintensity(
                repeats_data,
                marker_loc_dict
            )

Combined plot saved to: CHIP_REPEATS_NEW/1/plots/SOX2_inner_DAPIIntensityDis.png
Combined plot saved to: CHIP_REPEATS_NEW/1/plots/BRA_mid_DAPIIntensityDis.png
Combined plot saved to: CHIP_REPEATS_NEW/1/plots/GATA3_outer_DAPIIntensityDis.png
Combined plot saved to: CHIP_REPEATS_NEW/2/plots/SOX2_inner_DAPIIntensityDis.png
Combined plot saved to: CHIP_REPEATS_NEW/2/plots/BRA_mid_DAPIIntensityDis.png
Combined plot saved to: CHIP_REPEATS_NEW/2/plots/GATA3_outer_DAPIIntensityDis.png
Combined plot saved to: CHIP_REPEATS_NEW/3/plots/SOX2_inner_DAPIIntensityDis.png
Combined plot saved to: CHIP_REPEATS_NEW/3/plots/BRA_mid_DAPIIntensityDis.png
Combined plot saved to: CHIP_REPEATS_NEW/3/plots/GATA3_outer_DAPIIntensityDis.png
